In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

In [2]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset = dataset["train"]  # Just take the training split for now

In [3]:
print(dataset.shape)

(8551, 3)


In [4]:
test_dataset = dataset[8001:]

In [5]:
train_dataset = dataset[:8000]

In [6]:
for i in range(50):
    print(dataset[i])

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0}
{'sentence': "One more pseudo generalization and I'm giving up.", 'label': 1, 'idx': 1}
{'sentence': "One more pseudo generalization or I'm giving up.", 'label': 1, 'idx': 2}
{'sentence': 'The more we study verbs, the crazier they get.', 'label': 1, 'idx': 3}
{'sentence': 'Day by day the facts are getting murkier.', 'label': 1, 'idx': 4}
{'sentence': "I'll fix you a drink.", 'label': 1, 'idx': 5}
{'sentence': 'Fred watered the plants flat.', 'label': 1, 'idx': 6}
{'sentence': 'Bill coughed his way out of the restaurant.', 'label': 1, 'idx': 7}
{'sentence': "We're dancing the night away.", 'label': 1, 'idx': 8}
{'sentence': 'Herman hammered the metal flat.', 'label': 1, 'idx': 9}
{'sentence': 'The critics laughed the play off the stage.', 'label': 1, 'idx': 10}
{'sentence': 'The pond froze solid.', 'label': 1, 'idx': 11}
{'sentence': 'Bill rolled out of the room.', 'label': 1, '

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
## treat 3letters as a single token
tokenized_data = tokenizer(train_dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(train_dataset["label"])  # Label is already an array of 0 and 1

In [8]:
# from transformers import TFAutoModelForSequenceClassification
# from tensorflow.keras.optimizers import Adam

# # Load and compile our model
# model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# # Lower learning rates are often better for fine-tuning transformers
# model.compile(optimizer=Adam(3e-5))  # No loss argument!

# model.fit(tokenized_data, labels)

In [9]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
for key,val in tokenized_data.items():
    print(key)

input_ids
token_type_ids
attention_mask


In [12]:
print(tokenized_data['input_ids'].shape)

(8000, 47)


In [13]:
print(tokenized_data['token_type_ids'].shape)

(8000, 47)


In [14]:
print(tokenized_data['attention_mask'].shape)

(8000, 47)


In [15]:
j = 1000

In [16]:
print(dataset[j])

{'sentence': 'I read three his books.', 'label': 0, 'idx': 1000}


In [17]:
print(tokenized_data['input_ids'][j])

[ 101  146 2373 1210 1117 2146  119  102    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [18]:
print(tokenized_data['token_type_ids'][j])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [19]:
print(tokenized_data['attention_mask'][j])

[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [20]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split( tokenized_data , labels , test_size= 0.15, random_state=42 )


##### Simple train test split is not  working because tokenization is a dictionary and it only works for arrays, so we have to split all the 3 keys of dictionary into separatae lists and merge them once the work is done

In [22]:
input_ids = tokenized_data['input_ids']
token_type_ids = tokenized_data['token_type_ids']
attention_mask = tokenized_data['attention_mask']
from sklearn.model_selection import train_test_split
X_train1, X_test1, Y_train1, Y_test1 = train_test_split( input_ids , labels , test_size= 0.15, random_state=42 )
X_train2, X_test2, Y_train2, Y_test2 = train_test_split( token_type_ids , labels , test_size= 0.15, random_state=42 )
X_train3, X_test3, Y_train3, Y_test3 = train_test_split( attention_mask , labels , test_size= 0.15, random_state=42 )

In [23]:
td = dict()
td['input_ids'] = X_train1
td['token_type_ids'] = X_train2
td['attention_mask'] = X_train3
test_td = dict()
test_td['input_ids'] = X_test1
test_td['token_type_ids'] = X_test2
test_td['attention_mask'] = X_test3
td_labels = Y_train1
td_test_labels = Y_test1

In [24]:
print(len(td['input_ids']))
print(len(test_td['input_ids']))
print(len(td_labels))
print(len(td_test_labels))

6800
1200
6800
1200


In [25]:
### I am training the model using my own script
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
start = time.time()
callback = tf.keras.callbacks.ModelCheckpoint(filepath='Bert.tf',
                                                  monitor='val_acc', mode='max',
                                                 verbose=1)
opt = Adam(learning_rate=0.000001)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = opt,
              metrics = ['accuracy'])
history = model.fit(td,
                    td_labels,
                    validation_split=0.2,
                    batch_size = 32,
                    epochs = 12,
                    callbacks=[callback])

Epoch 1/12
170/170 [==============================] - ETA: 0s - loss: 1.4363 - accuracy: 0.6835
Epoch 1: saving model to Bert.tf


INFO:tensorflow:Assets written to: Bert.tf\assets


INFO:tensorflow:Assets written to: Bert.tf\assets


170/170 [==============================] - 107s 569ms/step - loss: 1.4363 - accuracy: 0.6835 - val_loss: 0.6931 - val_accuracy: 0.7110
Epoch 2/12
170/170 [==============================] - ETA: 0s - loss: 0.7345 - accuracy: 0.6770
Epoch 2: saving model to Bert.tf


INFO:tensorflow:Assets written to: Bert.tf\assets


...

INFO:tensorflow:Assets written to: Bert.tf\assets


170/170 [==============================] - 108s 637ms/step - loss: 0.6960 - accuracy: 0.6634 - val_loss: 0.6931 - val_accuracy: 0.7110
Epoch 12/12
170/170 [==============================] - ETA: 0s - loss: 0.6948 - accuracy: 0.6507
Epoch 12: saving model to Bert.tf


INFO:tensorflow:Assets written to: Bert.tf\assets


INFO:tensorflow:Assets written to: Bert.tf\assets


170/170 [==============================] - 123s 725ms/step - loss: 0.6948 - accuracy: 0.6507 - val_loss: 0.6931 - val_accuracy: 0.7110


##### Testing (Inference)


In [45]:
for key,val in test_dataset.items():
    print(key,val)

sentence ["Evan's every idea was completely insane.", 'Sally is making scones, and Gillian is too.', 'Everyone claimed that the poison was neutralized.', 'Jonathan persuaded Kate to lick himself.', 'That Jason arrived infuriated Medea.', 'Medea was happy, because she had got the highest marks', 'Keep yourself clean!', 'Cassandra has foretold disaster again.', "Bill's reading of Shakespeare satisfied me", 'Who poisoned who?', 'Pigs love truffles.', 'Owners of pigs love truffles', 'So quickly did the vampire move, that we barely saw him.', 'Humans love to eat those pigs.', 'She has kissed she.', 'Jason intended for he to learn magic.', 'Jason persuaded Medea to be treated by the doctor', 'It is true that I might be doing something other than going to the party.', 'Jason expected Medea to be treated by the doctor', 'I found there.', 'Moya said she liked football.', 'Anson became the Mayor', 'Kane ate dirt.', 'Benjamin gave the cloak to Nathan', 'The fig chuckled', 'Poseidon had run away, 

In [38]:
# test_dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data_test = tokenizer(test_dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data_test = dict(tokenized_data_test)

labels_test = np.array(test_dataset["label"])  # Label is already an array of 0 and 1

In [39]:
y_pred = model.predict(tokenized_data_test)
print(y_pred)

18/18 [==============================] - 4s 69ms/step
TFSequenceClassifierOutput(loss=None, logits=array([[-0.54708064, -0.38874626],
       [-0.7011964 , -0.4900629 ],
       [-0.68425846, -0.47410753],
       ...,
       [-0.78460985, -0.47649327],
       [-0.5890778 , -0.43024975],
       [-0.6655205 , -0.47994596]], dtype=float32), hidden_states=None, attentions=None)


In [42]:
y_pred2 = np.round_(y_pred[0])
y_pred3 = np.argmax(y_pred2, axis = 1)

In [43]:
from sklearn.metrics import accuracy_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(labels_test, y_pred3)))


Accuracy: 0.59

